In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import tensorflow as tf
from collections import Counter
import json

In [5]:
cards = pd.read_json("../data/mtg/ModernCards.json").transpose()

In [8]:
cols = cards.columns
cols_to_include = ['colorIdentity', 'colors', 'convertedManaCost', 'manaCost', 'name',
                    'subtypes', 'supertypes', 'text', 'type', 'types',
                    'power', 'toughness', 'names', 'side', 'loyalty']
cards = cards[cols_to_include]
creatures = cards.loc[cards['type'].str.contains('Creature')]
creatures["num_colors"] = creatures["colors"].map(len)
mono_creatures = creatures.loc[creatures['num_colors'] <= 1]

MAIN GOAL: Determine a creature's color identity based on: (number of features)
- CMC (1)
- Power (1)
- Toughness (1)
- Type (boolean cols for each of the top 100 tribes) (100)
- Keywords (see keywords.json and list of evergreen keywords on https://mtg.fandom.com/wiki/Evergreen) (20-100)
- Name? (Would need a way to break this down (https://web.stanford.edu/group/pdplab/pdphandbook/handbookch8.html))


In [44]:
all_type_labels = Counter([x for l in mono_creatures['subtypes'] for x in l])
all_type_labels.most_common(100)
evergreen_keywords = ["Activate", "Attach", "Cast", "Counter", "Create", "Destroy", "Discard", "Exchange", "Exile", "Fight",
                       "Mill", "Play", "Reveal", "Sacrifice", "Scry", "Search", "Shuffle", "Tap", "Untap"]
my_common_words = ["Enchantment", "Artifact", "+1/+1", "Token", "Draw" "Land", "Nonland", "Spell", "Creature",]
evergreen_abilities = ["Deathtouch", "Defender", "Double Strike", "Enchant", "Equip", "First Strike", "Flash", "Flying",
                        "Haste", "Hexproof", "Indestructible", "Lifelink", "Menace", "Protection", "Reach", "Trample",
                          "Vigilance", "Ward", "Regenerate", "Shroud", "Intimidate", "Prowess"]


In [35]:
mono_creatures["f_cmc"] = (mono_creatures / 7.5) - 1  # [0,15] -> [-1, 1]

mono_creatures['f_pow'] = mono_creatures['power'].replace({"1+*": 1, "*": "0"}) # Assume all *'s are 0 (as per the rules)
mono_creatures['f_pow'] = ((mono_creatures['f_pow'].astype(int) + 1) / 9) - 1 # [-1,16] -> [-1, 1]

mono_creatures['f_tough'] = mono_creatures['toughness'].replace({"1+*": 1, "*": "0"}) # Assume all *'s are 0 (as per the rules)
mono_creatures['f_tough'] = ((mono_creatures['f_tough'].astype(int) + 1) / 9) - 1 # [-1,16] -> [-1, 1]

C:\Users\WDAmo\AppData\Local\Temp\ipykernel_11412\3413877128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mono_creatures['f_pow'] = mono_creatures['power'].replace({"1+*": 1, "*": "0"}) # Assume all *'s are 0 (as per the rules)
C:\Users\WDAmo\AppData\Local\Temp\ipykernel_11412\3413877128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mono_creatures['f_pow'] = ((mono_creatures['f_pow'].astype(int) + 1) / 9) - 1 # [-1,16] -> [-1, 1]
C:\Users\WDAmo\AppData\Local\Temp\ipykernel_11412\3413877128.py:3

-0.666667    1642
-0.777778    1527
-0.555556    1287
-0.444444     830
-0.333333     477
-0.222222     228
-0.888889     163
-0.111111      95
 0.000000      43
 0.111111      18
 0.222222      15
 0.555556       7
 0.333333       6
 0.444444       4
 0.777778       2
 0.888889       1
-1.000000       1
Name: f_tough, dtype: int64

In [42]:
with open("../data/mtg/Keywords.json") as f:
    keywords = json.load(f)
keywords = keywords['abilityWords'] + keywords['keywordAbilities'] + keywords['keywordActions']
len(keywords)

217

In [43]:
cards.loc[cards['name'] == 'Glimpse the Unthinkable']

,colorIdentity,colors,convertedManaCost,manaCost,name,subtypes,supertypes,text,type,types,power,toughness,names,side,loyalty
Glimpse the Unthinkable,"[B, U]","[B, U]",2.0,{U}{B},Glimpse the Unthinkable,[],[],Target player puts the top ten cards of their ...,Sorcery,[Sorcery],NaN,NaN,NaN,NaN,NaN
